In [1]:
import pandas as pd
import pyodbc
from sqlalchemy import create_engine
import mysql.connector
from datetime import datetime, timedelta

In [2]:
db_config = {
    'host': 'Localhost',
    'user': 'root',
    'password': '',
    'database': 'bhavDB',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [3]:
start_date="17012024"
end_date="17012024"
dates=[]
current_date = datetime.strptime(start_date,"%d%m%Y")
end_dt = datetime.strptime(end_date,"%d%m%Y")
while current_date<=end_dt:
    dates.append(current_date.strftime("%d%m%Y"))
    current_date+=timedelta(days=1)

for date in dates:
    day_of_week = datetime.strptime(date,"%d%m%Y").weekday()
    if day_of_week in [5,6]:
        continue

    data = pd.read_csv(f"/Users/vinayak/Desktop/IIT Roorkee/Downloading Bhavcopy/bhav{date}.csv")
    df = pd.DataFrame(data)

    df['DATE1'] = pd.to_datetime(df['DATE1'], format='%d-%b-%Y').dt.strftime('%Y-%m-%d')
    for index, row in df.iterrows():
        cursor.execute("""
            INSERT INTO StockData (SYMBOL, SERIES, DATE1, PREV_CLOSE, OPEN_PRICE, HIGH_PRICE, LOW_PRICE, LAST_PRICE, CLOSE_PRICE, AVG_PRICE, TTL_TRD_QNTY, TURNOVER_LACS, NO_OF_TRADES, DELIV_QTY, DELIV_PER)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, tuple(row))
    df.to_sql('StockData', con=engine, if_exists='append', index=False)

In [4]:
connection.commit()
cursor.close()
connection.close()